# Non-correspondences

In [30]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/home/yoyee/Documents/pytorch-dense-correspondence/dense_correspondence


In [31]:
%matplotlib inline
# import dense_correspondence_manipulation.utils.utils as utils
# utils.add_dense_correspondence_to_python_path()
import correspondence_plotter
import correspondence_finder
# from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
import os
import numpy as np
import torch
import time


In [99]:
def uvto1d(points, H):
    # assert points.dim == 2
    return points[:,0]*H + points[:,1]

H, W = 30, 40
uv_a_matches = (torch.tensor(np.array([1]), dtype=torch.float32), 
                torch.tensor(np.array([1]), dtype=torch.float32))
matches_a = uv_a_matches[0]*H + uv_a_matches[1]
matches_a = uvto1d(uv_a_matches, H)
print("uv_a_matches: ", uv_a_matches)
print("matches_a: ", matches_a)

uv_b_matches = (torch.tensor(np.array([1]), dtype=torch.float32), 
                torch.tensor(np.array([1]), dtype=torch.float32))
matches_b = uvto1d(uv_b_matches, H)

img_b_shape = (H, W)
img_a_shape = img_b_shape

# image_a_pred


TypeError: tuple indices must be integers or slices, not tuple

In [57]:
# num_attempts = 5

# img_a_index = dataset.get_random_image_index(scene)
# img_a_rgb, img_a_depth, _, img_a_pose = dataset.get_rgbd_mask_pose(scene, img_a_index)

# img_b_index = dataset.get_img_idx_with_different_pose(scene, img_a_pose, num_attempts=50)
# img_b_rgb, img_b_depth, _, img_b_pose = dataset.get_rgbd_mask_pose(scene, img_b_index)

# img_a_depth_numpy = np.asarray(img_a_depth)
# img_b_depth_numpy = np.asarray(img_b_depth)

# start = time.time()
# uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth_numpy, img_a_pose, 
#                                                                     img_b_depth_numpy, img_b_pose,
#                                                                     num_attempts=num_attempts,
#                                                                     device='CPU')


start = time.time()
# uv_b_non_matches = correspondence_finder.create_non_correspondences(uv_b, img_a_depth_numpy.shape, num_non_matches_per_match=10)
uv_b_non_matches = correspondence_finder.create_non_correspondences(uv_b_matches, img_b_shape, num_non_matches_per_match=10, img_b_mask=None)
print  (time.time() - start, "seconds for non-matches")
if uv_b_non_matches is not None:
    print (uv_b_non_matches[0].shape)

    import torch
    # This just checks to make sure nothing is out of bounds
    print (torch.min(uv_b_non_matches[0]))
    print (torch.min(uv_b_non_matches[1]))
    print (torch.max(uv_b_non_matches[0]))
    print (torch.max(uv_b_non_matches[1]))
    
#     fig, axes = correspondence_plotter.plot_correspondences_direct(img_a_rgb, img_a_depth_numpy, img_b_rgb, img_b_depth_numpy, uv_a, uv_b, show=False)
#     uv_a_long = (torch.t(uv_a[0].repeat(3, 1)).contiguous().view(-1,1), torch.t(uv_a[1].repeat(3, 1)).contiguous().view(-1,1))
#     uv_b_non_matches_long = (uv_b_non_matches[0].view(-1,1), uv_b_non_matches[1].view(-1,1) )
#     correspondence_plotter.plot_correspondences_direct(img_a_rgb, img_a_depth_numpy, img_b_rgb, img_b_depth_numpy, uv_a_long, uv_b_non_matches_long, use_previous_plot=(fig,axes),
#                                                   circ_color='r')


0.002017498016357422 seconds for non-matches
torch.Size([1, 10])
tensor(1.)
tensor(4.)
tensor(36.)
tensor(28.)


In [60]:
uv_a_non_matches = correspondence_finder.create_non_correspondences(uv_a_matches, img_a_shape, num_non_matches_per_match=10, img_b_mask=None)
uv_b_non_matches = correspondence_finder.create_non_correspondences(uv_b_matches, img_b_shape, num_non_matches_per_match=10, img_b_mask=None)

non_matches_a = uvto1d(uv_a_non_matches, H)
non_matches_b = uvto1d(uv_b_non_matches, H)

print("non_matches_a: ", non_matches_a)

# uv_b_non_matches # (u, v)


non_matches_a:  tensor([[ 862.,  286., 1089.,  478.,  977.,  337.,  390.,  951.,   12.,  132.]])


In [61]:
torch.tensor(np.array([1]), dtype=torch.float32)

tensor([1.])

In [63]:
from loss_functions.pixelwise_contrastive_loss import PixelwiseContrastiveLoss


In [44]:
def pltImshow(img):
    from matplotlib import pyplot as plt
    plt.imshow(img)
    plt.show()

In [83]:
D = 3
# image_a = torch.tensor(np.random.rand(H, W), dtype=torch.float32)
# image_b = torch.tensor(np.random.rand(H, W), dtype=torch.float32)

image_a_pred = torch.tensor(np.random.rand(1, H*W, D), dtype=torch.float32)
image_b_pred = torch.tensor(np.random.rand(1, H*W, D), dtype=torch.float32)

# pltImshow(image_a.numpy())
# pltImshow(image_b.numpy())
# print("image_a_pred: ", ))



In [85]:
alpha = 0.5

match_loss, matches_a_descriptors, matches_b_descriptors = \
    PixelwiseContrastiveLoss.match_loss(image_a_pred, image_b_pred, matches_a.long(), matches_b.long())
non_match_loss, num_hard_negatives, non_matches_a_descriptors, non_matches_b_descriptors = \
    PixelwiseContrastiveLoss.non_match_descriptor_loss(image_a_pred, image_b_pred, 
                                                   non_matches_a.long().squeeze(), non_matches_b.long().squeeze())


print("match_loss: ", match_loss)
print("matches_a_descriptors: ", matches_a_descriptors.shape)
print("matches_b_descriptors: ", matches_b_descriptors.shape)

print("non_match_loss: ", non_match_loss)
print("num_hard_negatives: ", num_hard_negatives)
print("non_matches_a_descriptors: ", non_matches_a_descriptors.shape)

# loss, match_loss, non_match_loss = \
#     PixelwiseContrastiveLoss.get_triplet_loss(image_a_pred,
#                                         image_b_pred,
#                                         matches_a.long(),
#                                         matches_b.long(),
#                                         non_matches_a.long(),
#                                         non_matches_b.long(), 
#                                         alpha = alpha)

match_loss:  tensor(0.1347)
matches_a_descriptors:  torch.Size([1, 1, 1, 3])
matches_b_descriptors:  torch.Size([1, 1, 1, 3])
non_match_loss:  tensor([0.0000, 0.0000, 0.0000, 0.1419, 0.0010, 0.0422, 0.0000, 0.0000, 0.0000,
        0.0832])
num_hard_negatives:  4
non_matches_a_descriptors:  torch.Size([10, 3])


In [86]:
uv_a_masked_long, uv_b_masked_non_matches_long = \
        self.create_non_matches(matches_1, matches_2_masked_non_matches, self.num_masked_non_matches_per_match)

NameError: name 'self' is not defined

In [104]:
def scale_homography(H, shape, shift=(-1,-1)):
    height, width = shape[0], shape[1]
    trans = np.array([[2./width, 0., shift[0]], [0., 2./height, shift[1]], [0., 0., 1.]])
    H_tf = np.linalg.inv(trans) @ H @ trans
    return H_tf

homographies = np.identity(3)[np.newaxis,:,:]
height, width = 240, 320
image_shape = np.array([height, width])

homographies_H = np.stack([scale_homography(homographies, image_shape, shift=(-1,-1)) for H in homographies])

print("homographies: ", homographies)
print("homographies_H: ", homographies_H)


homographies:  [[[1. 0. 0.]
  [0. 1. 0.]
  [0. 0. 1.]]]
homographies_H:  [[[[1. 0. 0.]
   [0. 1. 0.]
   [0. 0. 1.]]]]


In [105]:
H, W = 240, 320
cell_size = 8
Hc, Wc = H//cell_size, W//cell_size

device = 'cpu'

def get_coor_cells(Hc, Wc, cell_size, device='cpu'):
    coor_cells = torch.stack(torch.meshgrid(torch.arange(Hc), torch.arange(Wc)), dim=2)
    coor_cells = coor_cells.type(torch.FloatTensor).to(device)
#     coor_cells = coor_cells * cell_size + cell_size // 2
    ## coord_cells is now a grid containing the coordinates of the Hc x Wc
    ## center pixels of the 8x8 cells of the image
    return coor_cells.view(-1, 2)

coor_cells = get_coor_cells(Hc, Wc, cell_size=cell_size, device=device)
print("coor_cells: ", coor_cells)
print("coor_cells: ", coor_cells.shape)

def warp_coor_cells_with_homographies(coor_cells, homographies):
    # warped_coor_cells = warp_points(coor_cells.view([-1, 2]), homographies, device)
#     warped_coor_cells = normPts(coor_cells.view([-1, 2]), shape)
    warped_coor_cells = torch.stack((warped_coor_cells[:,1], warped_coor_cells[:,0]), dim=1) # (y, x) to (x, y)
    warped_coor_cells = warp_points(warped_coor_cells, homographies, device)

    warped_coor_cells = torch.stack((warped_coor_cells[:, :, 1], warped_coor_cells[:, :, 0]), dim=2)  # (batch, x, y) to (batch, y, x)

    shape_cell = torch.tensor([H//cell_size, W//cell_size]).type(torch.FloatTensor).to(device)
    # warped_coor_mask = denormPts(warped_coor_cells, shape_cell)

#     warped_coor_cells = denormPts(warped_coor_cells, shape)
    return warped_coor_cells

matches_a = uvto1d(coor_cells, Hc)
print("matches_a: ", matches_a.shape)

matches_b = matches_a
print("matches_b: ", matches_b.shape)


coor_cells:  tensor([[ 0.,  0.],
        [ 0.,  1.],
        [ 0.,  2.],
        ...,
        [29., 37.],
        [29., 38.],
        [29., 39.]])
coor_cells:  torch.Size([1200, 2])
matches_a:  torch.Size([1200])
matches_b:  torch.Size([1200])


In [107]:
match_loss, matches_a_descriptors, matches_b_descriptors = \
    PixelwiseContrastiveLoss.match_loss(image_a_pred, image_b_pred, matches_a.long(), matches_b.long())
print("match_loss: ", match_loss)
print("matches_a_descriptors: ", matches_a_descriptors.shape)
print("matches_b_descriptors: ", matches_b_descriptors.shape)

match_loss:  tensor(0.4838)
matches_a_descriptors:  torch.Size([1, 1200, 3])
matches_b_descriptors:  torch.Size([1, 1200, 3])


In [ ]:
def create_non_matches(self, uv_a, uv_b_non_matches, multiplier):
    """
    Simple wrapper for repeated code
    :param uv_a:
    :type uv_a:
    :param uv_b_non_matches:
    :type uv_b_non_matches:
    :param multiplier:
    :type multiplier:
    :return:
    :rtype:
    """
    uv_a_long = (torch.t(uv_a[0].repeat(multiplier, 1)).contiguous().view(-1, 1),
                 torch.t(uv_a[1].repeat(multiplier, 1)).contiguous().view(-1, 1))

    uv_b_non_matches_long = (uv_b_non_matches[0].view(-1, 1), uv_b_non_matches[1].view(-1, 1))

    return uv_a_long, uv_b_non_matches_long

In [117]:
def descriptor_loss_sparse(descriptors, descriptors_warped, homographies, mask_valid=None,
                           cell_size=8, device='cpu', descriptor_dist=4, lamda_d=250,
                           num_matching_attempts=1000, num_masked_non_matches_per_match=10, **config):
    """
    consider batches of descriptors
    :param descriptors:
        Output from descriptor head
        tensor [batch_size, descriptors, Hc, Wc]
    :param descriptors_warped:
        Output from descriptor head of warped image
        tensor [batch_size, descriptors, Hc, Wc]
    """
    batch_size, Hc, Wc = descriptors.shape[0], descriptors.shape[2], descriptors.shape[3]
    
    image_a_pred = descriptors.view(batch_size, -1, Hc*Wc).transpose(1,2)  # torch [batch_size, H*W, D]
    print("image_a_pred: ", image_a_pred.shape)
    image_b_pred = descriptors_warped.view(batch_size, -1, Hc*Wc).transpose(1,2)  # torch [batch_size, H*W, D]
    print("image_a_pred: ", image_a_pred.shape)

    # matches
    ## sample points (implement sampling later)
    ## warp points
        # coor_cell
    uv_a = get_coor_cells(Hc, Wc, cell_size)
        # warp coor_cell
#     homographies_H = 
    homographies_H = np.stack([scale_homography(homographies, image_shape, shift=(-1,-1)) for H in homographies])
    
    uv_b_matches = warp_coor_cells_with_homographies(uv_a, homographies)
    matches_a = uvto1d(uv_a, H)
    print("matches_a: ", matches_a.shape)
    matches_b = uvto1d(uv_b_matches, H)
    print("matches_b: ", matches_b.shape)
    
    # filtering!!!
    
    ## select from descriptor
    ## calculate matches loss
    match_loss, matches_a_descriptors, matches_b_descriptors = \
        PixelwiseContrastiveLoss.match_loss(image_a_pred, image_b_pred, matches_a.long(), matches_b.long())
    
    # non matches
    ## sample non matches
    img_b_shape = (Hc, Wc)
    uv_b_non_matches = correspondence_finder.create_non_correspondences(uv_b_matches, img_b_shape, num_non_matches_per_match=10, img_b_mask=None)
    
    ## create_non_correspondences
    uv_a_masked_long, uv_b_masked_non_matches_long = \
        self.create_non_matches(matches_1, matches_2_masked_non_matches, self.num_masked_non_matches_per_match)
    
    ## non matches loss
    non_match_loss, num_hard_negatives, non_matches_a_descriptors, non_matches_b_descriptors = \
    PixelwiseContrastiveLoss.non_match_descriptor_loss(image_a_pred, image_b_pred, 
                                                   non_matches_a.long().squeeze(), non_matches_b.long().squeeze())
    
    loss = lamda_d*match_loss + non_match_loss
    pass
    
descriptors = torch.tensor(np.random.rand(1, D, Hc, Wc), dtype=torch.float32)
print("descriptors: ", descriptors.shape)
descriptors_warped = torch.tensor(np.random.rand(1, D, Hc, Wc), dtype=torch.float32)
descriptor_loss_sparse(descriptors, descriptors_warped, homographies)

descriptors:  torch.Size([1, 3, 30, 40])
image_a_pred:  torch.Size([1, 1200, 3])
image_a_pred:  torch.Size([1, 1200, 3])


NameError: name 'normPts' is not defined

In [68]:
matches_a.long().type()

'torch.LongTensor'

In [ ]:
def compute_loss_on_dataset(dcn, data_loader, loss_config, num_iterations=500,):
    """

    Computes the loss for the given number of iterations

    :param dcn:
    :type dcn:
    :param data_loader:
    :type data_loader:
    :param num_iterations:
    :type num_iterations:
    :return:
    :rtype:
    """
    dcn.eval()

    # loss_vec = np.zeros(num_iterations)
    loss_vec = []
    match_loss_vec = []
    non_match_loss_vec = []
    counter = 0
    pixelwise_contrastive_loss = PixelwiseContrastiveLoss(dcn.image_shape, config=loss_config)

    batch_size = 1

    for i, data in enumerate(data_loader, 0):

        # get the inputs
        data_type, img_a, img_b, matches_a, matches_b, non_matches_a, non_matches_b, metadata = data
        data_type = data_type[0]

        if len(matches_a[0]) == 0:
            print ("didn't have any matches, continuing")
            continue

        img_a = Variable(img_a.cuda(), requires_grad=False)
        img_b = Variable(img_b.cuda(), requires_grad=False)

        if data_type == "matches":
            matches_a = Variable(matches_a.cuda().squeeze(0), requires_grad=False)
            matches_b = Variable(matches_b.cuda().squeeze(0), requires_grad=False)
            non_matches_a = Variable(non_matches_a.cuda().squeeze(0), requires_grad=False)
            non_matches_b = Variable(non_matches_b.cuda().squeeze(0), requires_grad=False)

        # run both images through the network
        image_a_pred = dcn.forward(img_a)
        image_a_pred = dcn.process_network_output(image_a_pred, batch_size)

        image_b_pred = dcn.forward(img_b)
        image_b_pred = dcn.process_network_output(image_b_pred, batch_size)

        # get loss
        if data_type == "matches":
            loss, match_loss, non_match_loss = \
                pixelwise_contrastive_loss.get_loss(image_a_pred,
                                                    image_b_pred,
                                                    matches_a,
                                                    matches_b,
                                                    non_matches_a,
                                                    non_matches_b)



            loss_vec.append(loss.data[0])
            non_match_loss_vec.append(non_match_loss.data[0])
            match_loss_vec.append(match_loss.data[0])


        if i > num_iterations:
            break

    loss_vec = np.array(loss_vec)
    match_loss_vec = np.array(match_loss_vec)
    non_match_loss_vec = np.array(non_match_loss_vec)

    loss = np.average(loss_vec)
    match_loss = np.average(match_loss_vec)
    non_match_loss = np.average(non_match_loss_vec)

    return loss, match_loss, non_match_loss
